# **USEFUL LIBRARIES**

In [3]:
import pandas as pd
import numpy as np
import  urllib.request
import urllib.parse
from bs4 import BeautifulSoup


import tinydb
from langdetect import detect
from pymed import PubMed

import requests
import re
import random
import glob
import os
import time
import pickle
np.random.seed(42)

# **UTILITY METHODS**

* get_publication_ID
* create_list_of_keywords
* text2BoW

In [4]:
def extract_publication_info(publication):
    info = publication.toDict() #each element of the papers list is a dictionary class
    ID = info["pubmed_id"]
    if len(ID)>10:
        ID = ID.split('\n')
        ID = ID[0]
    title = info['title']
    if 'authors' in info:
      authors = info['authors']
    else:
      authors = []
    if 'keywords' in info:
      keywords = info['keywords']
    else:
      keywords = []
    if 'journal' in info:
      journal = info['journal']
    else:
      journal = []
    if 'doi' in info:
      doi = info['doi']
    else:
      doi = []

    return ID, title, authors, keywords, journal, doi


def create_list_of_keywords(filename, keyword_or_evidence = 'e'):
  fp = open(filename, 'r')
  lines = fp.readlines()
  kw_list = []

  for line in lines:
    if keyword_or_evidence == 'e':
      x = line.split('|')
      x = x[1]
      x = x.split('\n')
    else:
      x = line.split('\n')
    kw_list.append(x[0])
  
  fp.close()
  return kw_list

def text2BoW(processed_docs):

  # Converting text to Bag of Words (BoW)
  dictionary = gensim.corpora.Dictionary(processed_docs)
  # Remove very rare and very common words
  #dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n = 100000)
  # Create the Bag-of-words for each abstract reporting how many words 
  # and how many times those words appear
  bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

  return dictionary, bow_corpus

def study_type_keywords(file_path):

  studytype_dict = {}

  # Go to all the folders that have names from 1 to 7
  for file_name in range(1,8):
    file_path_2 = os.path.join(file_path, str(file_name))
    all_txt_files = os.listdir(file_path_2)

    # Go through all txt files in the folder
    for txt_file in all_txt_files:

      # Take the name without .txt extension
      txt_file_name = txt_file[:-4]
      # If this key doesn't exist add it and assign it with an empty list
      if txt_file_name not in studytype_dict:
        studytype_dict[txt_file_name] = []

      txt_file_path = os.path.join(file_path_2, txt_file)

      # Read the lines from one .txt file
      with open(txt_file_path) as f:
        lines = f.readlines()
      
      # Take one line and append it to the list
      for line in lines:
        if line[:2] == '--':           # Some .txt files start with --SMTH--
          continue
        # Remove '\n' from each line
        line = line[:-2]
        # If the keyword doesn't exist in the list add it
        if line not in studytype_dict[txt_file_name]:
          studytype_dict[txt_file_name].append(line.lower())

  return studytype_dict
  


# **PUBLICATION CLASS**

This class containes following methods:
* display_abstract()
* count_overall_occuracies()
* KeyWords_found_in_abstract()
* number_of_different_keywords
* study_quality

The publication is defined by its **url** address. We are using only the publications' **abstracts** since they are always available in **PubMed** database. We have two textual files that contain **keywords** which we used to score the publication according to its connection with our interest domain. We would like to give higher scores to the articles that are about serious applications for kids.   

In [22]:
class Publication:
  
  def __init__(self, in_url):

    """ Takes URL of the publication, takes its abstract in the original 
        format, extracts only the useful text from the abstract.  """ 
    
    self.url = in_url

    # Given the URL we extract the abstract in the original format
    try:
      page = urllib.request.urlopen(self.url)
      time.sleep(6)
    except requests.exceptions.ConnectionError:
      r.status_code = "Connection refused"
    
    soup = BeautifulSoup(page, 'lxml')
    article_abs = soup.find(id = "abstract")

    # Sometimes we can't find the publication -> print '-'
    try:
      self.abstract = article_abs.get_text()
    except AttributeError:
      self.abstract = ''
      print('-')

    # Take only the useful text from the abstract -> remove its subparts, '\n' signs... 
    self.extract_abstract_text()

    ####################### Check if this is useful ############################
    self.EvidenceLevel = create_list_of_keywords('keywords_studydesign.txt')
    self.KeyWords = create_list_of_keywords('keywords_context.txt', 'k')
    ############################################################################

  def display_abstract(self): 
    print(self.abstract)

 
  def count_overall_occurancies(self):

    """ Takes our manually selected keywords and counts their occurances 
        in the abstract. Returns the sum of all occurances.  """
    
    abs_txt = self.abstract
    # Convert all big letters into low letters
    abs_txt = abs_txt.lower()  
    sum_txt = 0
    for word in self.KeyWords:
        count = abs_txt.count(word)
        sum_txt += count
    return(sum_txt)
  
  
  def KeyWords_found_in_abstract(self):
    
    """ Takes our manually selected keywords and checks their existence in 
        the abstract. Returns the list of founded keywords.  """
  
    abs_txt = self.abstract
    kw = []
    for word in self.KeyWords:
        if word in abs_txt.lower():
            kw.append(word)
    return(kw)

  
  def number_of_different_keywords(self):
    return(len(self.KeyWords_found_in_abstract()))

  
  def study_quality(self):
    abs_txt = self.abstract
    abs_txt = abs_txt.lower()
    sd = []
    for word in self.EvidenceLevel:
      word_low = word.lower()
      if word in abs_txt or word_low in abs_txt:
        sd.append(word)
    return(sd)

  
  def study_score(self):
    total_num = self.number_of_different_keywords()
    sd = self.study_quality()
    score = round(len(sd)/(total_num + 1),5)
    return score

  
  def manual_score(self):
    total_num = self.number_of_different_keywords()
    kw = self.KeyWords_found_in_abstract()
    score = round(len(kw)/(total_num + 1),5)
    return score

  
  def extract_abstract_text(self):

    #print('Abstract before: \n')
    #print(self.abstract)
    #print('\n')

    # Remove all \n signs
    self.abstract = self.abstract.replace('\n', '') 
    # Remove the Abstract word in the beginning of each article
    self.abstract = self.abstract.replace('Abstract', '')
    # Remove multiple spaces if they exist
    self.abstract = " ".join(self.abstract.split())

    # Remove the Keywords section
    self.abstract = re.sub(r"(?i)(?:Keywords:).*?[.?!]", '', self.abstract)

    # Find subparts such are Conclusion, Methods... and remove them
    subparts = self.find_article_subparts()
    for s in subparts:
      self.abstract = self.abstract.replace(s, '')

    # Remove all types of url addresses
    self.abstract = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', self.abstract)

    # Remove the space before the first string
    self.abstract.lstrip()
    
    #print('Abstract after: \n')
    #print(self.abstract)

  def get_keywords(self):
    self.keywords = re.findall(r"(?i)(?:Keywords:).*?[.?!]", self.abstract)
    if len(self.keywords)==0:
      self.keywords = []
    else:
      self.keywords = self.keywords[0]
      self.keywords = self.keywords.split(';')
      self.keywords[0] = self.keywords[0].replace('Keywords:', '')
    return self.keywords

  def find_article_subparts(self):
    subparts = re.findall(r'(\w+:)', self.abstract)
    return subparts

 


# **PUBLICATIONS CLASS**

In [6]:
class PubMedPublications:

   def __init__(self, number_of_articles = 10,             
                tool = 'ehealth_group14', 
                pubmedurlbase = "https://pubmed.ncbi.nlm.nih.gov/",
                email = "iva97.ja@gmail.com"):
    
    self.tool = tool
    self.email = email
    self.num_articles =  number_of_articles
    self.pubmedurlbase = pubmedurlbase

    self.pubmed = PubMed(tool = self.tool, email = self.email)
    self.abstracts = []
    self.article_dict = {}
    self.abs_IDs = set()

   def search(self, query_words):

     # Initialize empty lists for all info
     IDs = []
     titles = []
     kws = []
     journals = []
     dois = []
     authorss = []
     
     for query_word in query_words:

       # Get best N articles for given query
       try:
         n_article_results = self.pubmed.query(query_word, max_results = self.num_articles)  
       except TypeError:
         pass
       time.sleep(10)
      
       for one_article_result in n_article_results:
 
         # Get article INFO and abstract text
         article_ID, title, authors, keywords, journal, doi = extract_publication_info(one_article_result)

         # Do this only if this ID is new
         if article_ID not in self.abs_IDs:
           self.abs_IDs.add(article_ID)
           IDs.append(article_ID)
           titles.append(title)
           kws.append(keywords)
           journals.append(journal)
           dois.append(doi)
           authorss.append(authors)

           article_abstract = Publication(self.pubmedurlbase + article_ID + "/")
           

           ########################## Check this part ############################

           # Occurance of all keywords in the article 
           all_kw_occurance = article_abstract.count_overall_occurancies()
           # Keywords that appear in the article
           kw = article_abstract.KeyWords_found_in_abstract()
           # Number of keywords that don't appear in the article
           nb_diff = article_abstract.number_of_different_keywords()
           # Article score
           sd = article_abstract.manual_score()
 
           #######################################################################

           # Dictionary that conatines abstract ID and its score
           if article_ID not in self.article_dict:
             self.article_dict[article_ID] = sd

             # Add that abstract text to the global list of abstracts
             self.abstracts.append(article_abstract.abstract)
     
     return self.abstracts, IDs, titles, kws, journals, dois, authorss




# **TOPIC MODELING WITH LDA** - *Latent Dirichlet Allocation*

Since we extracted the publications for all of the games from our database, we can use their abstracts, preprocess them and insert them into the LDA model. We should expect couple of classes (we will define them) that will explain the topic of the particular publication. We would like to have a class that will contain the keywords such are kids, education, serious...

In [7]:
import nltk
nltk.download('wordnet')
from nltk import tokenize
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

from tqdm.notebook import tqdm
tqdm.pandas()

import gensim
from gensim import corpora, models
from gensim.utils import tokenize

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [8]:
class TextPreprocessing:

  def __init__(self, abs_text):
    self.abs_text = abs_text

  def preprocess(self):
    self.split_abstract_into_words()    
    self.remove_short_words()   
    self.remove_stop_words()   
    self.lemmatize_stemming()

    return self.words_final_form

  def split_abstract_into_words(self):
    self.tokenized_words = list(gensim.utils.tokenize(self.abs_text))

  def remove_short_words(self, thresh = 3):
    self.words = []
    for word in self.tokenized_words:
      if len(word)>=thresh:
        self.words.append(word)


  def remove_stop_words(self):
    self.words_without_stop = []
    for w in self.words:
      if w not in gensim.parsing.preprocessing.STOPWORDS.union(set(['the', 'thi'])):
        self.words_without_stop.append(w)


  def lemmatize_stemming(self):
    stemmer = PorterStemmer()
    self.words_final_form = []
    for w in self.words_without_stop:
      new_w = stemmer.stem(WordNetLemmatizer().lemmatize(w, pos='v'))
      self.words_final_form.append(new_w)


In [15]:
pubmedurlbase = "https://pubmed.ncbi.nlm.nih.gov/"

# Read da file with games with their publications 
with open('pubs_per_game.pickle', 'rb') as f:
    data_file = pickle.load(f)

In [ ]:
# Preprocessing the abstracts extracted
processed_docs = []
processed_docs_with_id = {}


for name, pubs in data_file.items():
    for id in pubs['ID']:
        article_abstract = Publication(pubmedurlbase + id + "/")
        article_abstract = article_abstract.abstract
        if article_abstract=='':
            continue
        processing_abs = TextPreprocessing(article_abstract)
        result = processing_abs.preprocess()
        processed_docs.append(result)

            

In [9]:
SAVE = False
if SAVE==True:
    # Save all preprocessed abstract texts
    with open('processed_docs.pickle', 'wb') as handle:
        pickle.dump(processed_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save all preprocessed abstract texts
with open('processed_docs.pickle', 'rb') as f:
    processed_docs = pickle.load(f)


In [10]:
# Create a dictionary from 'processed_docs' containing the number of times a 
# word appears in all of the abstracts (training dataset)
dictionary, bow_corpus = text2BoW(processed_docs)


# Define the LDA model - suppose the number of topics (here it is 8)
LDA_model =  gensim.models.LdaMulticore(bow_corpus, 
                                        num_topics = 4, 
                                        id2word = dictionary,                                    
                                        passes = 10,
                                        workers = 2)

# For each topic, we will explore the words occuring in that topic 
# and its relative weight
for idx, topic in LDA_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.012*"the" + 0.009*"studi" + 0.005*"group" + 0.005*"effect" + 0.005*"base" + 0.005*"model" + 0.005*"particip" + 0.005*"data" + 0.004*"differ" + 0.004*"game"


Topic: 1 
Words: 0.011*"the" + 0.010*"cell" + 0.008*"patient" + 0.006*"studi" + 0.004*"protein" + 0.004*"activ" + 0.004*"effect" + 0.004*"tumor" + 0.003*"treatment" + 0.003*"result"


Topic: 2 
Words: 0.011*"the" + 0.008*"patient" + 0.007*"studi" + 0.005*"associ" + 0.004*"covid" + 0.004*"popul" + 0.004*"health" + 0.004*"data" + 0.004*"risk" + 0.004*"year"


Topic: 3 
Words: 0.010*"the" + 0.005*"studi" + 0.004*"increas" + 0.004*"result" + 0.003*"differ" + 0.003*"chang" + 0.003*"effect" + 0.003*"patient" + 0.003*"gene" + 0.003*"measur"




## **VISUALISATION**

In [11]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [12]:
vis = gensimvis.prepare(topic_model=LDA_model, corpus=bow_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


In [13]:
pyLDAvis.save_html(vis, 'lda2.html')
pyLDAvis.save_json(vis, 'lda2.js')

# **TOPIC CLASSIFICATION**


*   Class 1 - *Serious games*
*   Class 2 - *Brain activity analysis*
*   Class 3 - *Health effects on study analysis*
*   Class 4 - *Health treatment experiment analysis*



In [23]:
import http.client


pub_topic = {}
for name, pubs in data_file.items():
    for id in pubs['ID']:
        try:
            article_abstract = Publication(pubmedurlbase + id + "/")
        except (http.client.IncompleteRead) as e:
             article_abstract = e.partial

        article_abstract = article_abstract.abstract
        if article_abstract=='':
            continue
        processing_abs = TextPreprocessing(article_abstract)
        result = processing_abs.preprocess()
        bow_vector = dictionary.doc2bow(result)
        pub_topic[id] = {'0': 0.0, '1': 0.0, '2': 0.0, '3': 0.0}

        for index, score in sorted(LDA_model[bow_vector], key=lambda tup: -1*tup[1]):
            pub_topic[id][str(index)]=score   
            
with open('pubs_topics.pickle', 'wb') as handle:
    pickle.dump(pub_topic, handle, protocol=pickle.HIGHEST_PROTOCOL)       


-
-
-


# **STUDY TYPE CLASSIFICATION**

Given the "*Study Type Dictionary*" folder which containes keywords for each of the following 8 study types: 
* *MetaAnalysis.txt*
* *ObservationalStudy.txt*
* *RCT.txt*
* *SystematicReview.txt*
* *CaseControl.txt*
* *CaseSeries.txt*
* *CohortStudy.txt*
* *Other.txt*


There are *7* folders where each contains aforementioned textual files and many of keywords are repeat when comparing e.g. the *MetaAnalysis.txt* from the folders *1* and *2*. The idea is to go through all of this folders and files and gather unique keywords that will represnt each one of the study types.Looking at the provided keys, they are already lemmatized. Therefore, we don't need to preprocess them too.


In [ ]:
#!unzip "Study type dictionaries"

path = "StudyType_Dictionaries"
study_type_dictionary = study_type_keywords(path)

# The keys in the dictonary
print('Dictionary keys:')
print(study_type_dictionary.keys(), '\n')

# List of keywords for one key e.g. MetaAnalysis
print('MetaAnalysis keywords:')
print(study_type_dictionary['MetaAnalysis'],'\n')

# Number of classes
print('Number of study types: {}'.format(len(study_type_dictionary.keys())))

## *Publications for DataBase Apps*

Now, we want to go through all of the games we have in our database. Taking the games' names we will query the PubMed database and extract couple of publications for each one of them. When the publications are collected, they will be preprocessed as we did before in the code. Finally, in each of the abstract we will count the occurances of keywords for each study plan independently. The result will be the perentage of abstract belonging to one of the classes that represent particular study type. The abstract will be assigned with a class for which it has the highest probability of belonging to.

In [ ]:
from tinydb import TinyDB
db = TinyDB('Ourdatabase.json')

In [ ]:
def getFieldData(fieldName, db):
  result = [r[fieldName] for r in db]
  return result

game_names = getFieldData('title', db)
print('Game names: \n', game_names)

Since the game names are not very suitable for querying we need to preprocess them. Firstly, they usually contain more words and we can notice two main parts of the name (one before signs **' - '** or **' : '**, another after them). Some of them are writen in the language different from English. These are some of the examples:

* *Learn 33 Languages Free - Mondly*
* *Sololearn: Learn to Code (Python, Javascript, etc)*
* *U-Dictionary*
* *ANTON: Kindergarten - Grade 5*
* *Language Learning - Spanish, Korean, French & More*
* *Meritnation: CBSE, ICSE & more (Free Live Classes)*
* *تعليم اللغة الانجليزية من الصفر بالصوت والصورة*
* *wifistudy - #1 Exam Preparation, Free Mock Tests*
* *Exam Preparation App: Live Class | Mock Test | PYP*



Since there are names that are not in English, first, we will check the language. If the language is English in that case we can split the name into shorter textual parts. Otherwise, we won't modify the name. Now, we need to decide how to split the names as those parts as queries too.

In [ ]:
queries_dict = {}

for name in game_names:

  if name not in queries_dict:
    queries_dict[name] = []

  # First add the whole name
  queries_dict[name].append(name)
  if name.isnumeric()==True:
    continue
  if detect(name) != 'en':
    queries_dict[name].append(name)
  
  else:
    # Split the name when ' - ' is present
    sub_names = name.split(' - ') 
    queries_dict[name].extend(sub_names)
    continue

    # Split the name when ':' is present
    sub_names = name.split(':') 
    queries_dict[name].extend(sub_names)
    continue

    # Split the name when '|' is present
    sub_names = name.split('|') 
    queries_dict[name].extend(sub_names)
    continue


# Unique list elements
for key in queries_dict:
  queries_dict[key] = list(set(queries_dict[key]))


In [ ]:
def count_occurances_study_type_keywords(abstract, keywords_dict):
  
  """ Given one abstract in semi-processed format and the Study type 
      keywords dictiory previously obtained """
  
  result_dict = {}
  counts_total = 0
  for study_type in keywords_dict:
    counts = 0
    for word in keywords_dict[study_type]:
      counts += abstract.count(word)
    counts_total += counts
    result_dict[study_type] = counts

  result_dict = {k: round(v/(counts_total+0.01), 4) for k, v in result_dict.items()}

  return result_dict

In [ ]:
from urllib.error import HTTPError

dict_game_score = {}
num_pubs_per_game = {}
dict_game_papers = {}

# Go through all the games 
for name in game_names:
  all_publications = PubMedPublications(number_of_articles = 10)
  # Take queries for one particular game
  queries = queries_dict[name]
  # Gather the abstracts
  abstracts, IDs, titles, kws, journals, dois, authorss  = all_publications.search(query_words = queries)
  # Extract information (ID, Name, Authors, Year, Study type, DOI, keywords)
  data = {}
  data['ID'] = IDs
  data['Title'] = titles
  data['Keywords'] = kws
  data['Journal'] = journals
  data['DOI'] = dois
  data['Authors'] = authorss

  df = pd.DataFrame.from_dict(data)
  dict_game_papers[name] = df

  #print(name, len(abstracts))
  num_pubs_per_game[name] = len(abstracts)
  results_abs = []
  for abstract in abstracts:
    result = count_occurances_study_type_keywords(abstract.lower(), study_type_dictionary)
    results_abs.append(result)
  
  dict_game_score[name] = results_abs


In [ ]:
dict_game_papers['Learn 33 Languages Free - Mondly']

,ID,Title,Keywords,Journal,DOI,Authors
0,34863443,Improving the accuracy of stroke clinical codi...,"[Activity-based funding, Casemix, Clinical cod...",Journal of clinical neuroscience : official jo...,10.1016/j.jocn.2021.10.024,"[{'lastname': 'Bacchi', 'firstname': 'Stephen'..."
1,34328444,Analyzing Patient Secure Messages Using a Fast...,"[FHIR, annotated corpus, data model, patient p...",Journal of medical Internet research,10.2196/26770,"[{'lastname': 'De', 'firstname': 'Amrita', 'in..."
2,34175663,Temporal lobe epilepsy: A never-ending story.,"[Automatism, Focal unaware seizures, Ictal sem...",Epilepsy & behavior : E&B,10.1016/j.yebeh.2021.108122,"[{'lastname': 'Abarrategui', 'firstname': 'Bel..."
3,34058634,Number and type of toys affect joint attention...,"[Joint attention, Mother-infant interaction, N...",Infant behavior & development,10.1016/j.infbeh.2021.101589,"[{'lastname': 'Koşkulu', 'firstname': 'Sümeyye..."
4,34009134,Effective German and English Language mHealth ...,"[English, German, apps, asthma, mobile health,...",JMIR mHealth and uHealth,10.2196/24907,"[{'lastname': 'Franzmair', 'firstname': 'Julia..."
5,33496673,Use of the Systematized Nomenclature of Medici...,"[SNOMED CT, natural language processing, scopi...",Journal of medical Internet research,10.2196/24594,"[{'lastname': 'Gaudet-Blavignac', 'firstname':..."
6,33336635,Associations of visual paired associative lear...,"[CANTAB, MoCA, PAL, aging, memory, neuropsycho...",International psychogeriatrics,10.1017/S1041610220003841,"[{'lastname': 'Hicks', 'firstname': 'Eleanora ..."
7,33252349,Machine Learning Electronic Health Record Iden...,"[Electronic Health Records, Gradient Boosting,...",JMIR medical informatics,10.2196/23930,"[{'lastname': 'Maarseveen', 'firstname': 'Tjar..."
8,32416320,Night shifts as a learning experience among nu...,"[Clinical education, Cross-sectional, Europe, ...",Nurse education today,10.1016/j.nedt.2020.104441,"[{'lastname': 'Dobrowolska', 'firstname': 'Bea..."
9,32318897,A Hybrid Reporting Platform for Extended RadLe...,"[Database, Medical informatics, Natural langua...",Journal of digital imaging,10.1007/s10278-020-00342-0\n10.1007/s13244-018...,"[{'lastname': 'Jungmann', 'firstname': 'Floria..."


In [ ]:
import pickle
with open('dict_game_scores.pickle', 'wb') as handle:
    pickle.dump(dict_game_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('number_pubs_per_game.pickle', 'wb') as handle:
    pickle.dump(num_pubs_per_game, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pubs_per_game.pickle', 'wb') as handle:
    pickle.dump(dict_game_papers, handle, protocol=pickle.HIGHEST_PROTOCOL)